# Movie Recommendation System

!["Image of Donald Draper in a Movie Theater"](images/dondraperinmovietheater.jpg)

## Overview

This project aims to develop a recommendation system that provides personalized movie recommendations based on user ratings. Utilizing the [MovieLens dataset](https://web.archive.org/web/20240828133414/https://grouplens.org/datasets/movielens/latest/) from the GroupLens research lab at the University of Minnesota, the model will be trained on a subset of the dataset containing 100,000 user ratings. The primary objective is to build a machine learning model that can accurately recommend the top 5 movies to a user, based on their ratings of other movies. This system can be valuable for streaming platforms and movie enthusiasts, offering tailored movie suggestions to enhance user experience and engagement.

The project will involve several steps, including data cleaning, exploratory data analysis, feature engineering, model selection, and evaluation. 

Throughout this project, we will also explore the relationships between different variables and their impact on movie recommendations. This will help us gain insights into user preferences and identify potential areas for improvement. Overall, this project has the potential to provide valuable insights and practical applications for the entertainment industry. By developing a recommendation system that can accurately suggest movies, streaming platforms can better engage their users, improve customer satisfaction, and increase viewership.

## Business Understanding

The entertainment industry, particularly streaming platforms, is highly competitive, with companies constantly striving to enhance user engagement and satisfaction. One of the major challenges faced by these platforms is providing personalized content recommendations that keep users engaged and reduce churn rates.

According to recent studies, personalized recommendations can significantly increase user engagement and satisfaction, leading to higher retention rates and increased viewership. This highlights the need for a robust recommendation system that can accurately suggest movies based on user preferences. By building a recommendation system that can provide top 5 movie recommendations to users based on their ratings of other movies, streaming platforms can offer a more tailored viewing experience.

The business value of this project lies in its ability to help streaming platforms improve their content recommendation strategies, increase user satisfaction, and reduce churn rates. By developing a recommendation system that can accurately suggest movies, platforms can better engage their users, leading to increased viewership and subscription renewals. This can provide a competitive edge in the highly competitive entertainment industry, ultimately driving revenue growth and customer loyalty.

In [1]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
sns.set_style('whitegrid')

from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
from surprise import Reader, Dataset

## Reading the Data

In [2]:
# Loading the data
movies = pd.read_csv('data/movies.csv')
ratings = pd.read_csv('data/ratings.csv')
tags = pd.read_csv('data/tags.csv')

## Tidying the data (ratings df)

1. Check data types and figure out which figures are numerical and which are categorical.
2. Check for null values.
3. Check for duplicate values

I check for null values in the dataset. There are none.

In [12]:
# checking for null values
ratings.isnull().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

I then check for duplicates. There are none.

In [13]:
# check for duplicates
ratings.duplicated().sum()

0

In [14]:
# check data types
ratings.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

In [15]:
# Drop the timestamp column
ratings.drop('timestamp', axis=1, inplace=True)

In [17]:
# extracting the title from the `movies` df based on the `movieId`
ratings_with_titles = ratings.merge(movies[['movieId', 'title']], on='movieId', how='left')

In [18]:
ratings_with_titles.head()

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,1,3,4.0,Grumpier Old Men (1995)
2,1,6,4.0,Heat (1995)
3,1,47,5.0,Seven (a.k.a. Se7en) (1995)
4,1,50,5.0,"Usual Suspects, The (1995)"


I'm starting with the `ratings` df for my basic recommendation model because it contains key user-item interactions, which are essential for collaborative filtering. This dataframe shows which users rated which movies and their ratings, allowing me to identify patterns in preferences.

I'll use collaborative filtering, focusing on either:

- User-Based: Recommending movies based on the preferences of similar users.
- Item-Based: Suggesting movies based on similarities between items that users have liked.

Using `ratings` gives me a solid foundation for identifying these relationships, and I can later integrate `movies` and `tags` dataframes for more nuanced recommendations.

Now, I'll create a scikit surprise dataset from the `ratings` dataframe.

In [ ]:
# reading the values as scikit surprise dataset
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)